In [1]:
import os
import pickle
import time
from importlib import reload

import pandas as pd
import numpy as np
import pandas as pd
import sys

from functools import reduce
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from typing import Tuple

In [2]:
import sys 
pwd = os.getcwd()
par_dir = os.path.abspath(os.path.join(pwd,os.pardir))
tools_dir = os.path.join(par_dir,'python')
print(tools_dir)
if tools_dir not in sys.path:
    sys.path.append(tools_dir) # go to parent dir from customFunctions import *

/mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/python


In [44]:
print(sys.path)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/azure_ml', '/anaconda/envs/premier_analyis/lib/python38.zip', '/anaconda/envs/premier_analyis/lib/python3.8', '/anaconda/envs/premier_analyis/lib/python3.8/lib-dynload', '', '/anaconda/envs/premier_analyis/lib/python3.8/site-packages', '/anaconda/envs/premier_analyis/lib/python3.8/site-packages/IPython/extensions', '/home/azureuser/.ipython', '/anaconda/envs/premier_analyis/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/premier_analyis/lib/python3.8/site-packages/azureml/automl/core/_vendor', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/python/tools/', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/python/tools/', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/python/tools', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-s

In [3]:
import tools.multi as tm
import tools.preprocessing as tp


In [4]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [5]:
# Unit of time to use for aggregation
TIME_UNIT = 'dfi'

# Whether to limit the output to folks with at least 1 COVID visit
COVID_ONLY = True

# Setting the file directories
#pwd = os.path.abspath(os.path.dirname(__file__))
pwd = os.getcwd()
prem_dir = os.path.join(pwd, "..", "data", "data", "")
out_dir = os.path.join(pwd, "..", "output", "")
parq_dir = os.path.join(out_dir, "parquet", "")
pkl_dir = os.path.join(out_dir, "pkl", "")
samp_dir = os.path.join(out_dir, "samples", "")

[os.makedirs(path, exist_ok=True) for path in [parq_dir, pkl_dir, samp_dir]]

[None, None, None]

In [6]:
pq = tp.load_parquets(prem_dir)

In [7]:
# Replacing NaN with 0
pq.id.dropna(axis=0, subset=['days_from_index'], inplace=True)

# Making some lookup tables to use later
medrec_dict = dict(
        zip(pq.id.pat_key.astype(int), pq.id.medrec_key.astype(int)))

day_dict = dict(
        zip(pq.id.pat_key.astype(int), pq.id.days_from_index.astype(int)))
covid_dict = dict(
        zip(pq.id.pat_key.astype(int), pq.id.covid_visit.astype(int)))



In [8]:
pq.vitals

,pat_key,observation_day_number,observation_time_of_day,lab_test,test_result_numeric_value
0,1348863696,1.0,22:58:00,Body weight:Mass:Pt: Patient:Qn (LOINC concept),104.60
1,1348863696,1.0,23:12:00,Body height:Len:Pt: Patient:Qn: (LOINC concept),180.34
2,1348863696,1.0,23:11:00,Body weight:Mass:Pt: Patient:Qn: (LOINC concept),104.60
3,1348863696,1.0,23:14:00,Body weight:Mass:Pt: Patient:Qn: (LOINC concept),104.60
4,1348863696,1.0,22:58:00,Body weight:Mass:Pt: Patient:Qn: (LOINC concept),104.60
...,...,...,...,...,...
74878660,1275646324,2.0,12:33:00,Body temperature:Temp:Pt:^Patient:Qn (LOINC co...,97.70
74878661,1275646324,2.0,12:33:00,Intravascular mean:Pres:Pt:Arterial system:Qn ...,92.00
74878662,1275646324,1.0,22:54:00,Body temperature:Temp:Pt: Patient:Qn (LOINC co...,97.70
74878663,1275646324,1.0,21:30:00,Breaths:NRat:Pt:Respiratory system:Qn (LOINC c...,17.00


In [9]:
print('Converting the free-text fields to features...')

# Vectorizing the single free text fields
vitals, v_dict = tp.df_to_features(
        pq.vitals,
        feature_prefix='vtl',
        text_col='lab_test',
        time_cols=['observation_day_number', 'observation_time_of_day'],
        num_col='test_result_numeric_value')

Converting the free-text fields to features...


In [10]:
vitals

,pat_key,ftr,observation_day_number,observation_time_of_day
0,1348863696,vtl0,1.0,22:58:00
1,1348863696,vtl1,1.0,23:12:00
2,1348863696,vtl2,1.0,23:11:00
3,1348863696,vtl2,1.0,23:14:00
4,1348863696,vtl2,1.0,22:58:00
...,...,...,...,...
74878660,1275646324,vtl147,2.0,12:33:00
74878661,1275646324,vtl104,2.0,12:33:00
74878662,1275646324,vtl85,1.0,22:54:00
74878663,1275646324,vtl26,1.0,21:30:00


In [11]:
v_dict

{'vtl0': 'Body weight:Mass:Pt: Patient:Qn (LOINC concept) q3.0',
 'vtl1': 'Body height:Len:Pt: Patient:Qn: (LOINC concept) q4.0',
 'vtl2': 'Body weight:Mass:Pt: Patient:Qn: (LOINC concept) q3.0',
 'vtl3': 'Body height:Len:Pt: Patient:Qn (LOINC concept) q4.0',
 'vtl4': 'Intravascular diastolic:Pres:Pt:Arterial system:Qn: (LOINC concept) q1.0',
 'vtl5': 'Intravascular systolic:Pres:Pt:Arterial system:Qn: (LOINC concept) q1.0',
 'vtl6': 'Heart rate:NRat:Pt:XXX:Qn: (LOINC concept) q1.0',
 'vtl7': 'Breaths:NRat:Pt:Respiratory system:Qn: (LOINC concept) q2.0',
 'vtl8': 'Intravascular systolic:Pres:Pt:Arterial system:Qn: (LOINC concept) q2.0',
 'vtl9': 'Intravascular diastolic:Pres:Pt:Arterial system:Qn: (LOINC concept) q0.0',
 'vtl10': 'Body temperature:Temp:Pt: Patient:Qn: (LOINC concept) q3.0',
 'vtl11': 'Body temperature:Temp:Pt: Patient:Qn: (LOINC concept) q2.0',
 'vtl12': 'Heart rate:NRat:Pt:XXX:Qn: (LOINC concept) q2.0',
 'vtl13': 'Breaths:NRat:Pt:Respiratory system:Qn: (LOINC concept)

In [62]:
os.path.join(par_dir,'output/samples','agg_samp.csv')

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/samples/agg_samp.csv'

In [73]:
flat_feature_sample = pd.read_csv(os.path.join(par_dir,'output/samples','agg_samp.csv'))
flat_feature_sample


,medrec_key,pat_key,dfi,vitals,bill,genlab,lab_res,proc,diag,covid_visit
0,-228917059,1195383165,2147483648,NaN,NaN,NaN,NaN,NaN,dx7417 dx7417 dx17589 dx18731 dx17589 dx18731,0
1,-227571988,1335947807,2147483648,NaN,NaN,NaN,NaN,NaN,dx1898 dx663 dx853,0
2,-223588856,1310892584,1,NaN,bill30 bill14 bill3 bill18 bill118 bill28 bill...,NaN,NaN,NaN,NaN,1
3,-223588856,1310892584,2,NaN,bill11 bill73 bill9 bill10 bill4 bill1653 bill...,NaN,NaN,NaN,NaN,1
4,-223588856,1310892584,3,NaN,bill1604 bill1697 bill1733 bill1678 bill2406 b...,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
9716,2054278425,1337006433,294,vtl45 vtl58 vtl48 vtl59 vtl28 vtl53 vtl26 vtl4...,bill22 bill6 bill2164 bill2164 bill2012 bill16...,genl147 genl68 genl409 genl8 genl192 genl9 gen...,NaN,NaN,NaN,1
9717,2054278425,1337006433,295,vtl30 vtl48 vtl30 vtl85 vtl48 vtl48 vtl61 vtl5...,bill6 bill22 bill1968 bill2102 bill1909 bill15...,genl734 genl147 genl681 genl668 genl988 genl73...,NaN,proc8,NaN,1
9718,2054278425,1337006433,296,vtl36 vtl48 vtl53 vtl22 vtl43 vtl48 vtl58 vtl5...,bill107 bill6 bill22 bill1632 bill1577 bill164...,genl147 genl366 genl39 genl30 genl138 genl8 ge...,NaN,NaN,NaN,1
9719,2054278425,1337006433,297,vtl61 vtl43 vtl48 vtl72 vtl22 vtl80 vtl48 vtl7...,bill33 bill1642 bill1651 bill1590 bill2297 bil...,NaN,lbrs417,proc6,NaN,1


In [74]:
ftr_cols = ['vitals', 'bill', 'genlab', 'lab_res', 'proc', 'diag']
# Ensure we're sorted
flat_feature_sample.sort_values(["medrec_key", "dfi"], inplace=True)
flat_feature_sample['ftrs'] = flat_feature_sample[ftr_cols].astype(str).replace(["None", "nan"], "").agg(" ".join, axis=1)

In [75]:
flat_feature_sample

,medrec_key,pat_key,dfi,vitals,bill,genlab,lab_res,proc,diag,covid_visit,ftrs
0,-228917059,1195383165,2147483648,NaN,NaN,NaN,NaN,NaN,dx7417 dx7417 dx17589 dx18731 dx17589 dx18731,0,dx7417 dx7417 dx17589 dx18731 dx17589 dx1...
1,-227571988,1335947807,2147483648,NaN,NaN,NaN,NaN,NaN,dx1898 dx663 dx853,0,dx1898 dx663 dx853
2,-223588856,1310892584,1,NaN,bill30 bill14 bill3 bill18 bill118 bill28 bill...,NaN,NaN,NaN,NaN,1,bill30 bill14 bill3 bill18 bill118 bill28 bil...
3,-223588856,1310892584,2,NaN,bill11 bill73 bill9 bill10 bill4 bill1653 bill...,NaN,NaN,NaN,NaN,1,bill11 bill73 bill9 bill10 bill4 bill1653 bil...
4,-223588856,1310892584,3,NaN,bill1604 bill1697 bill1733 bill1678 bill2406 b...,NaN,NaN,NaN,NaN,1,bill1604 bill1697 bill1733 bill1678 bill2406 ...
...,...,...,...,...,...,...,...,...,...,...,...
9716,2054278425,1337006433,294,vtl45 vtl58 vtl48 vtl59 vtl28 vtl53 vtl26 vtl4...,bill22 bill6 bill2164 bill2164 bill2012 bill16...,genl147 genl68 genl409 genl8 genl192 genl9 gen...,NaN,NaN,NaN,1,vtl45 vtl58 vtl48 vtl59 vtl28 vtl53 vtl26 vtl4...
9717,2054278425,1337006433,295,vtl30 vtl48 vtl30 vtl85 vtl48 vtl48 vtl61 vtl5...,bill6 bill22 bill1968 bill2102 bill1909 bill15...,genl734 genl147 genl681 genl668 genl988 genl73...,NaN,proc8,NaN,1,vtl30 vtl48 vtl30 vtl85 vtl48 vtl48 vtl61 vtl5...
9718,2054278425,1337006433,296,vtl36 vtl48 vtl53 vtl22 vtl43 vtl48 vtl58 vtl5...,bill107 bill6 bill22 bill1632 bill1577 bill164...,genl147 genl366 genl39 genl30 genl138 genl8 ge...,NaN,NaN,NaN,1,vtl36 vtl48 vtl53 vtl22 vtl43 vtl48 vtl58 vtl5...
9719,2054278425,1337006433,297,vtl61 vtl43 vtl48 vtl72 vtl22 vtl80 vtl48 vtl7...,bill33 bill1642 bill1651 bill1590 bill2297 bil...,NaN,lbrs417,proc6,NaN,1,vtl61 vtl43 vtl48 vtl72 vtl22 vtl80 vtl48 vtl7...


In [76]:
from sklearn.feature_extraction.text import CountVectorizer
ftrs = [doc for doc in flat_feature_sample.ftrs]
vec = CountVectorizer(ngram_range=(1, 1), min_df=5, binary=True)
vec.fit(ftrs)
vocab = vec.vocabulary_


In [77]:
int_ftrs = [[vocab[k] for k in doc.split() if k in vocab.keys()]
            for doc in ftrs]
flat_feature_sample["int_ftrs"] = int_ftrs

# list of integer sequence arrays split by medrec_key
int_seqs = [
    df.values for _, df in flat_feature_sample.groupby("medrec_key")["int_ftrs"]
]

# Converting to a nested list to keep things clean
seq_gen = [[seq for seq in medrec] for medrec in int_seqs]

In [81]:
int_seqs

[array([list([2697, 2697])], dtype=object),
 array([list([2427, 2668])], dtype=object),
 array([list([1385, 283, 1384, 539, 242, 1277, 1859, 1805, 722, 1878, 1657, 1892, 195, 1, 2, 1021, 315, 475, 395, 416, 1907, 1922, 2014, 1911, 1928, 1901, 1898, 1920]),
        list([195, 1864, 1892, 2, 1657, 395, 374, 452, 346, 420, 1044, 438, 475, 2070, 2014, 1974]),
        list([346, 438, 475, 420, 1044, 395, 374, 359, 1974, 2014]),
        list([1892, 195, 1864, 2, 1805, 1657, 420, 374, 359, 346, 438, 1044, 395, 1901, 2014, 1974]),
        list([1838, 1864, 891, 395, 374, 420, 311, 359, 438, 346, 311, 471, 1044, 1974, 2037, 1898, 1907, 1906, 2014]),
        list([1892, 1838, 1864, 420, 346, 438, 471, 311, 359, 1044, 1898, 1898, 1907, 1974, 2359, 2393, 2536, 2453, 2310, 2517, 2410, 2337, 2403, 2438, 2497])],
       dtype=object),
 array([list([2362])], dtype=object),
 array([list([])], dtype=object),
 array([list([2381, 2310, 2393, 2500, 2410, 2345])], dtype=object),
 array([list([2762, 2393])],